In [1]:
import numpy as np
from numpy.linalg import solve
import pandas as pd
import time
import sys
import csv
import math
import datetime
import itertools
import psutil
import random

In [2]:
def getTransitionProbability(N, R = 1):
    pr = np.zeros((R * N, R * N))
    for r in range(R):
        class_number = 0
        p = np.random.rand(N, N)
        for i, val in enumerate(np.sum(p, axis=1)):
            p[i] /= val
        for i in range(N):
            for j in range(N):
                pr[r * N+i,r * N+j] = p[i,j]
    return pr

In [3]:
def getEquivalence4(th, roop, p):
  reachable = np.zeros((len(p), len(p))) #全て0のpと同じ大きさの行列を用意
  equivalence = [[] for i in range(len(p))] #同値類を最大数用意
  list_number = 0

  for ix in range(roop): #n乗まで実施
    pn = np.linalg.matrix_power(p.copy(), ix+1) #累乗 
    for i in range(len(pn)):
      for j in range(len(pn)):
        if(pn[i][j] > th): #推移確率が閾値より大きいかチェック
          reachable[i,j] = 1

#  print('到達行列 = \n{0}'.format(reachable))
#  communicate = np.zeros((len(p), len(p))) #全て0のpと同じ大きさの行列を用意
#  for i in range(len(reachable)):
#    for j in range(len(reachable)):
#      if(reachable[i][j] == 1 and reachable[j][i] == 1):
    
  for i in range(len(pn)):
    for j in range(i+1, len(pn)):
      if(reachable[i][j] == 1 and reachable[j][i] == 1):
#        print('reachable {0} <-> {1}'.format(i, j))
        find = 0 #iでfind -> 1, jでfind -> 2
        idx = len(pn)
        for k in range(len(pn)):
          if i in equivalence[k]: #iが同値類k番目に存在している
            find = 1 #iは同値類k番目に存在
            idx = k
#            print('{0} find in {1}'.format(i, equivalence[k]))
            break
          elif j in equivalence[k]: 
            find = 2
            idx = k
#            print('{0} find in {1}'.format(j, equivalence[k]))
            break
        if find == 1:
          if j not in equivalence[idx]: #jは同値類kには存在しない (他のリストにもないことを確認する!!!他のリストにあった場合は移動がいい？ -> communicateがずれないように最後に集合で演算する)
            equivalence[idx].append(j) #jを追加
#            print('{0}に{1}を追加'.format(equivalence[idx], j))
#            print('リスト全体 {0}'.format(equivalence))     
            #break   
        elif find == 2:
          if i not in equivalence[idx]: #(他のリストにもないことを確認する!!!)
            equivalence[idx].append(i)
#            print('{0}に{1}を追加'.format(equivalence[idx], i))
#            print('リスト全体 {0}'.format(equivalence))         
            #break
        elif(find == 0): #新規に追加
          equivalence[list_number].append(i)
#          print('{0}に{1}を追加'.format(equivalence[list_number], i))
#          print('リスト全体 {0}'.format(equivalence))
          if(i != j):
            equivalence[list_number].append(j)
#            print('{0}に{1}を追加'.format(equivalence[list_number], j))
#            print('リスト全体 {0}'.format(equivalence))
          list_number += 1

  #4. Communicateにならないノードを登録
  for i in range(len(pn)):
    find = 0
    for j in range(len(pn)):
      if i in equivalence[j]:
        find = 1
        break
    if find == 0:
      equivalence[list_number].append(i)
#      print('Non Communication node {0} add'.format(i))
#      print('リスト全体 {0}'.format(equivalence))
      list_number += 1

  #5. エルゴード性の確認(class数が1のとき)
  class_number = 0
  for i in range(len(pn)):
    if len(equivalence[i]) > 0:
#      print('クラスの長さ : {0}, {1}'.format(len(equivalence[i]), equivalence[i]))
      class_number += 1

  for i in range(class_number):
    for j in range(i+1, class_number):
      s1 = set(equivalence[i])
      s2 = set(equivalence[j])
      if s1 & s2 :
#        print('重複 {0} & {1}'.format(i, j))
#        print('重複ノード {}'.format(s1 & s2))
        equivalence[i] = equivalence[i] + equivalence[j]
        equivalence[j].clear()

  #print('修正クラス数 {0}'.format(modify_class_number))
  for i in range(class_number):
  #  print(equivalence[i])
    equivalence[i] =list(set(equivalence[i]))

  #再度クラス数チェック
  class_number = 0
  for i in range(len(pn)):
    if len(equivalence[i]) > 0:
      class_number += 1
  
  #再度リストを構成
  modify_equivalence = [[] for i in range(class_number)] #同値類をクラス数用意
  l_index = 0
  for i in range(len(pn)):
    if len(equivalence[i]) > 0:
      modify_equivalence[l_index] = equivalence[i]
      l_index += 1

  return modify_equivalence, class_number, reachable

In [4]:
def getDegreeMax(equivalence, reachable):
  degree_max_value = [] #クラスの中で最大次数のものをクラス0から順番に入れる
  degree_max_index = []
  reachable_sum = np.sum(reachable, axis=0) + np.sum(reachable, axis=1) #到達行列から行和と列和の和
  for i in range(len(equivalence)): #クラス数だけ回す
    deg_max = -1 #最大値の初期値
    deg_max_index = -1 #最大値を持つノード番号
    for j in equivalence[i]: #ノード番号を一つずつ取り出す
      if deg_max < reachable_sum[j]:
        deg_max = reachable_sum[j]
        deg_index = j
    degree_max_value.append(deg_max)
    degree_max_index.append(deg_index)
  return degree_max_index, degree_max_value

In [5]:
def getNormalize(p, degree_max_index, p_plus = 0.01):
  for i in degree_max_index:
    for j in degree_max_index:
      if i != j: #(i,j),(j,i)両方に足す
        p[i][j] += p_plus 
  #推移確率に直す
  for i, val in enumerate(np.sum(p, axis=1)):
    p[i] /= val
  return p

In [6]:
def getAdjacency(reachable, equi, delta):
  adjacency = [[0 for i in range(len(equi))] for j in range(len(equi))]
  for i in range(len(equi)):
    for j in range(len(equi)):
      if reachable[equi[i]][equi[j]] >= delta:
        adjacency[i][j] = 1
  #print(adjacency)
  return adjacency

In [7]:
def getProximityCentrality(reachable, equivalence, delta):
  degree_max_value = [] #クラスの中で最大次数のものをクラス0から順番に入れる
  degree_max_index = []
  
  #reachable_sum = np.sum(reachable, axis=0) + np.sum(reachable, axis=1) #到達行列から行和と列和の和
  for i in range(len(equivalence)): #クラス数だけ回す
    if len(equivalence[i]) != 1:
      adjacency = getAdjacency(reachable,equivalence[i], delta)
      #print(adjacency)
      def_adjacency = adjacency.copy()
      new_adjacency = adjacency.copy()
      adcount = sum(1 for ai in range(len(adjacency)) for aj in range(len(adjacency)) if (adjacency[ai][aj] == 0) and (ai != aj)) 
      adroop = 1
      while adcount > 0:
        adroop += 1
        new_adjacency = (np.matrix(new_adjacency)*np.matrix(def_adjacency)).tolist()
        #print(new_adjacency,def_adjacency)
        for ai in range(len(adjacency)):
          for aj in range(len(adjacency)):
            if (adjacency[ai][aj] == 0) and (ai != aj) and (new_adjacency[ai][aj] != 0):
              adjacency[ai][aj] = adroop
        adcount = sum(1 for ai in range(len(adjacency)) for aj in range(len(adjacency)) if (adjacency[ai][aj] == 0) and (ai != aj))
      deg_max = 1000 #最大値の初期値
      deg_max_index = -1 #最大値を持つノード番号
      for j in range(len(equivalence[i])): #ノード番号を一つずつ取り出す
        if deg_max > sum(adjacency[j][aj] for aj in range(len(equivalence[i])) if aj != j):
          deg_max = sum(adjacency[j][aj] for aj in range(len(equivalence[i])) if aj != j)
          deg_index = equivalence[i][j]
        deg_max = (len(equivalence[i])-1)/ deg_max
    else:
      deg_max = 0
      deg_index = equivalence[i][0]
    degree_max_value.append(deg_max)
    degree_max_index.append(deg_index)
  return degree_max_index, degree_max_value
  #1隣接行列を作成, 変数定義(次数変数)

In [8]:
N = 33
delta = 1 / N * 1.5
#delta = 0.045
roop = 1
p_plus = 0.001
roop_num = 20
pr = getTransitionProbability(N)

In [16]:
#次数中心性で実施した場合
p = pr.copy()
class_number_list = [] #各回のクラス数を保存
degree_max_value_list = [] #各回のクラス毎の最大次数を保存
class_each_number_list = [[] for i in range(roop_num)] #各回のそれぞれのクラスの要素数
for i in range(roop_num):
  print('{0}回目'.format(i))
  equivalence, class_number, reachable = getEquivalence4(delta, roop, p)
  print('クラス数 : {0}'.format(class_number))
  class_number_list.append(class_number)
  for j in range(class_number):
    print(equivalence[j])
    class_each_number_list[i].append(len(equivalence[j]))
  degree_max_index, degree_max_value = getDegreeMax(equivalence, reachable)
  print('各クラスの最大次数を持つノード番号 : {0}'.format(degree_max_index))
  print('各クラスの最大次数 : {0}'.format(degree_max_value))
  degree_max_value_list.append(degree_max_value)
  p = getNormalize(p, degree_max_index)
  if class_number == 1:
    break
  print('\n')
print(class_number_list)
print(class_each_number_list)
print(degree_max_value_list)

0回目
クラス数 : 8
[0, 1, 3, 5, 7, 8, 9, 10, 11, 12, 13, 15, 16, 18, 19, 22, 23, 24, 25, 28, 30, 31, 32]
[2, 26, 14]
[27, 21]
[4]
[6]
[17]
[20]
[29]
各クラスの最大次数を持つノード番号 : [18, 26, 21, 4, 6, 17, 20, 29]
各クラスの最大次数 : [24.0, 18.0, 12.0, 7.0, 10.0, 12.0, 8.0, 14.0]


1回目
クラス数 : 7
[0, 1, 3, 5, 7, 8, 9, 10, 11, 12, 13, 15, 16, 18, 19, 22, 23, 24, 25, 28, 30, 31, 32]
[2, 26, 14]
[17, 29]
[27, 21]
[4]
[6]
[20]
各クラスの最大次数を持つノード番号 : [18, 14, 29, 21, 4, 6, 20]
各クラスの最大次数 : [23.0, 18.0, 16.0, 12.0, 8.0, 12.0, 8.0]


2回目
クラス数 : 2
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 28, 29, 30, 31, 32]
[27]
各クラスの最大次数を持つノード番号 : [18, 27]
各クラスの最大次数 : [23.0, 9.0]


3回目
クラス数 : 2
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 28, 29, 30, 31, 32]
[27]
各クラスの最大次数を持つノード番号 : [18, 27]
各クラスの最大次数 : [23.0, 9.0]


4回目
クラス数 : 2
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 28, 2

In [28]:
#データの保存
t_delta = datetime.timedelta(hours=9)
JST = datetime.timezone(t_delta, 'JST')
now = datetime.datetime.now(JST)
d = now.strftime('%Y%m%d%H%M%S')
path = '/content/drive/MyDrive/研究/Connection/Result/'
file = str(d)+'_Degree_N_'+str(N)+'_delta_'+str(delta)+'_p_plus_'+str(p_plus)+'_roop_num_'+str(roop_num)+'_class_number_list.csv'
np.savetxt(path + file, np.array(class_number_list), delimiter=",", fmt="%d")

file = str(d)+'_Degree_N_'+str(N)+'_delta_'+str(delta)+'_p_plus_'+str(p_plus)+'_roop_num_'+str(roop_num)+'_class_each_number_list.csv'
with open(path + file, 'w') as file:
    writer = csv.writer(file, lineterminator='\n')
    writer.writerows(class_each_number_list)

file = str(d)+'_Degree_N_'+str(N)+'_delta_'+str(delta)+'_p_plus_'+str(p_plus)+'_roop_num_'+str(roop_num)+'_degree_max_value_list.csv'
with open(path + file, 'w') as file:
    writer = csv.writer(file, lineterminator='\n')
    writer.writerows(degree_max_value_list)

file = str(d)+'_Degree_N_'+str(N)+'_delta_'+str(delta)+'_p_plus_'+str(p_plus)+'_roop_num_'+str(roop_num)+'_initial_p.csv'
np.savetxt(path + file, pr, delimiter=",", fmt="%f")

file = str(d)+'_Degree_N_'+str(N)+'_delta_'+str(delta)+'_p_plus_'+str(p_plus)+'_roop_num_'+str(roop_num)+'_last_p.csv'
np.savetxt(path + file, p, delimiter=",", fmt="%f")

In [20]:
#近接中心性で実施した場合
p = pr.copy()
for i in range(roop_num):
  print('{0}回目'.format(i))
  equivalence, class_number, reachable = getEquivalence4(delta, roop, p)
  print('クラス数 : {0}'.format(class_number))
  for i in range(class_number):
    print(equivalence[i])
  degree_max_index, degree_max_value = getProximityCentrality(reachable, equivalence, delta)
  print('各クラスの最大次数を持つノード番号 : {0}'.format(degree_max_index))
  print('各クラスの最大次数 : {0}'.format(degree_max_value))
  p = getNormalize(p, degree_max_index)
  if class_number == 1:
    break
  print('\n')

0回目
クラス数 : 9
[0, 2, 5, 7, 8, 9, 11, 12, 13, 15, 20, 22, 23, 26, 27, 29]
[32, 1, 3, 10, 13, 14, 19, 21, 23, 24, 26, 28, 31]
[4]
[6]
[16]
[17]
[18]
[25]
[30]
各クラスの最大次数を持つノード番号 : [8, 26, 4, 6, 16, 17, 18, 25, 30]
各クラスの最大次数 : [24.0, 0.5714285714285714, 0, 0, 0, 0, 0, 0, 0]


1回目
クラス数 : 8
[0, 2, 5, 7, 8, 9, 11, 12, 13, 15, 20, 22, 23, 26, 27, 29]
[32, 1, 21, 23]
[3, 10, 13, 14, 19, 24, 26, 28, 31]
[16, 25]
[17, 18]
[4]
[6]
[30]
各クラスの最大次数を持つノード番号 : [8, 21, 19, 16, 17, 4, 6, 30]
各クラスの最大次数 : [24.0, 3.0, 0.6153846153846154, 1.0, 1.0, 0, 0, 0]


2回目
クラス数 : 7
[0, 2, 3, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 19, 20, 22, 23, 24, 26, 27, 28, 29, 31]
[32, 1, 21, 23]
[16, 25]
[17, 18]
[4]
[6]
[30]
各クラスの最大次数を持つノード番号 : [9, 21, 16, 17, 4, 6, 30]
各クラスの最大次数 : [0.5789473684210527, 3.0, 1.0, 1.0, 0, 0, 0]


3回目
クラス数 : 6
[0, 2, 3, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 19, 20, 22, 23, 24, 26, 27, 28, 29, 31]
[32, 16, 17, 18, 21, 25]
[1]
[4]
[6]
[30]
各クラスの最大次数を持つノード番号 : [15, 17, 1, 4, 6, 30]
各クラスの最大次数 : [0.564102564

In [11]:
# ROSS P195 Example 4.14
# this Markov chain is irreducible
pr = np.array([[1/2, 1/2, 0], [1/2, 1/4, 1/4], [0, 1/3, 2/3]])
print(pr)
#delta = 1 / len(pr)
delta = 0
roop = 2
equivalence, class_number, reachable = getEquivalence4(delta, roop, pr)
print('クラス数 = {0}'.format(class_number))
for i in range(class_number):
  print('クラス ID = {0}'.format(i))
  print(equivalence[i])

print('到達行列 \n {0}'.format(reachable))

pr_n = np.linalg.matrix_power(pr, 30)
print(pr_n)
init = [ 1/ len(pr_n) for i in range(len(pr_n))]
pi = np.dot(init, pr_n)
print(pi)

[[0.5        0.5        0.        ]
 [0.5        0.25       0.25      ]
 [0.         0.33333333 0.66666667]]
クラス数 = 1
クラス ID = 0
[0, 1, 2]
到達行列 
 [[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]
[[0.36363654 0.36363641 0.27272705]
 [0.36363641 0.36363637 0.27272722]
 [0.36363607 0.36363629 0.27272764]]
[0.36363634 0.36363636 0.2727273 ]


In [12]:
#ROSS P196 Example 4.15
# The classes of this Markov chain are {0,1}, {2}, and {3}
pr = np.array([[1/2, 1/2, 0, 0], [1/2, 1/2, 0, 0], [1/4, 1/4, 1/4, 1/4], [0, 0, 0, 1]])
print(pr)
delta = 0
roop = 4
equivalence, class_number, reachable = getEquivalence4(delta, roop, pr)
print('クラス数 = {0}'.format(class_number))
for i in range(class_number):
  print('クラス ID = {0}'.format(i))
  print(equivalence[i])

print('到達行列 \n {0}'.format(reachable))

pr_n = np.linalg.matrix_power(pr, 30)
print(pr_n)
init = [ 1/ len(pr_n) for i in range(len(pr_n))]
pi = np.dot(init, pr_n)
print(pi)

[[0.5  0.5  0.   0.  ]
 [0.5  0.5  0.   0.  ]
 [0.25 0.25 0.25 0.25]
 [0.   0.   0.   1.  ]]
クラス数 = 3
クラス ID = 0
[0, 1]
クラス ID = 1
[2]
クラス ID = 2
[3]
到達行列 
 [[1. 1. 0. 0.]
 [1. 1. 0. 0.]
 [1. 1. 1. 1.]
 [0. 0. 0. 1.]]
[[5.00000000e-01 5.00000000e-01 0.00000000e+00 0.00000000e+00]
 [5.00000000e-01 5.00000000e-01 0.00000000e+00 0.00000000e+00]
 [3.33333333e-01 3.33333333e-01 8.67361738e-19 3.33333333e-01]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 1.00000000e+00]]
[3.33333333e-01 3.33333333e-01 2.16840434e-19 3.33333333e-01]


In [13]:
#ROSS P198 Example 4.16
# all states communicate.
pr = np.array([[0, 0, 1/2, 1/2], [1, 0, 0, 0], [0, 1, 0, 0], [0, 1, 0, 0]])
#この行列は3回以上で完全な到達となる。２回では間接的な到達
print(pr)
delta = 0
roop = 2
equivalence, class_number, reachable = getEquivalence4(delta, roop, pr)
print('クラス数 = {0}'.format(class_number))
for i in range(class_number):
  print('クラス ID = {0}'.format(i))
  print(equivalence[i])

print('到達行列 \n {0}'.format(reachable))

[[0.  0.  0.5 0.5]
 [1.  0.  0.  0. ]
 [0.  1.  0.  0. ]
 [0.  1.  0.  0. ]]
クラス数 = 1
クラス ID = 0
[0, 1, 2, 3]
到達行列 
 [[0. 1. 1. 1.]
 [1. 0. 1. 1.]
 [1. 1. 0. 0.]
 [1. 1. 0. 0.]]


In [14]:
# Ross P198 Example 4.17
# This chain consists of the three classes {0,1}, {2,3}, and {4}
pr = np.array([[1/2, 1/2, 0, 0, 0], [1/2, 1/2, 0, 0, 0], [0, 0, 1/2, 1/2, 0], [0, 0, 1/2, 1/2, 0], [1/4, 1/4, 0, 0, 1/2]])
print(pr)
delta = 0
roop = 10
equivalence, class_number, reachable = getEquivalence4(delta, roop, pr)
print('クラス数 = {0}'.format(class_number))
for i in range(class_number):
  print('クラス ID = {0}'.format(i))
  print(equivalence[i])

print('到達行列 \n {0}'.format(reachable))

[[0.5  0.5  0.   0.   0.  ]
 [0.5  0.5  0.   0.   0.  ]
 [0.   0.   0.5  0.5  0.  ]
 [0.   0.   0.5  0.5  0.  ]
 [0.25 0.25 0.   0.   0.5 ]]
クラス数 = 3
クラス ID = 0
[0, 1]
クラス ID = 1
[2, 3]
クラス ID = 2
[4]
到達行列 
 [[1. 1. 0. 0. 0.]
 [1. 1. 0. 0. 0.]
 [0. 0. 1. 1. 0.]
 [0. 0. 1. 1. 0.]
 [1. 1. 0. 0. 1.]]
